<a href="https://colab.research.google.com/github/Michaelzats/Fundamental-Analysis-of-the-asset/blob/main/Fundamental%20Analysis%20of%20the%20asset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fundamental Analysis of the asset

The following tool helps to quickly find all the relevant fundamental information with regards to the put asset (Stocks, crypto etc).

Such information as Fundamental Ratios, Recent News and Recent Insider Trades with their volumes and costs will be displayed 

Required libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

Setting up the tool

In [2]:
pd.set_option('display.max_colwidth', 25)

# Input
symbol = input('Enter a ticker: ')
print ('Getting data for ' + symbol + '...\n')

# Set up scraper
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

def get_fundamentals():
    try:
        # Find fundamentals table
        fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]
        
        # Clean up fundamentals dataframe
        fundamentals.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
        colOne = []
        colLength = len(fundamentals)
        for k in np.arange(0, colLength, 2):
            colOne.append(fundamentals[f'{k}'])
        attrs = pd.concat(colOne, ignore_index=True)
    
        colTwo = []
        colLength = len(fundamentals)
        for k in np.arange(1, colLength, 2):
            colTwo.append(fundamentals[f'{k}'])
        vals = pd.concat(colTwo, ignore_index=True)
        
        fundamentals = pd.DataFrame()
        fundamentals['Attributes'] = attrs
        fundamentals['Values'] = vals
        fundamentals = fundamentals.set_index('Attributes')
        return fundamentals

    except Exception as e:
        return e
    
def get_news():
    try:
        # Find news table
        news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
        links = []
        for a in html.find_all('a', class_="tab-link-news"):
            links.append(a['href'])
        
        # Clean up news dataframe
        news.columns = ['Date', 'News Headline']
        news['Article Link'] = links
        news = news.set_index('Date')
        return news

    except Exception as e:
        return e

def get_insider():
    try:
        # Find insider table
        insider = pd.read_html(str(html), attrs = {'class': 'body-table'})[0]
        
        # Clean up insider dataframe
        insider = insider.iloc[1:]
        insider.columns = ['Trader', 'Relationship', 'Date', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']
        insider = insider[['Date', 'Trader', 'Relationship', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']]
        insider = insider.set_index('Date')
        return insider

    except Exception as e:
        return e


Enter a ticker: TSLA
Getting data for TSLA...



Extracting the data

In [3]:

print ('Fundamental Ratios: ')
print(get_fundamentals())

print ('\nRecent News: ')
print(get_news())

print ('\nRecent Insider Trades: ')
print(get_insider())


Fundamental Ratios: 
                 Values
Attributes             
Index           S&P 500
Market Cap      772.87B
Income            8.40B
Sales            62.19B
Book/sh           32.96
...                 ...
ATR               63.07
Volatility  8.54% 7.02%
Prev Close       728.00
Price            769.59
Change            5.71%

[72 rows x 1 columns]

Recent News: 
                              News Headline              Article Link
Date                                                                 
May-13-22 10:31PM  Elon Musk Sows Doubt ...  https://finance.yahoo...
09:49PM            Dow Jones Futures: St...  https://www.investors...
06:26PM            EV Battery Technology...  https://www.investors...
05:24PM            Elon Musk says Twitte...  https://finance.yahoo...
05:15PM            Elon Musk invited leg...  https://finance.yahoo...
...                                     ...                       ...
05:10PM            Rivian earnings provi...  https://finance.yahoo...